<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Ingeniería de Características</h1>

<h3>Datos categóricos</h3>



<p> Julio Waissman Vilanova </p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/tipos/python/cat.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

</center>

## Tutorial para el manejo de datos categóricos en Pandas

Pandas es una librería de Python que proporciona herramientas poderosas para trabajar con datos estructurados. Los datos categóricos son un tipo de datos que representan categorías o grupos, como género, color o estado civil. En este tutorial, exploraremos cómo manejar datos categóricos en Pandas.

### 1. Conversión a tipo categórico
Pandas proporciona la clase Categorical para representar datos categóricos. Puedes convertir una columna de tu DataFrame a tipo categórico usando el método astype:

In [ ]:
import numpy as np
import pandas as pd

data = {'genero': ['Masculino', 'Femenino', 'Masculino', 'Femenino'],
        'edad': [25, 30, 28, 22],
        'ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Sevilla']}

df = pd.DataFrame(data)

df['genero'] = df['genero'].astype('category')
print(df.dtypes)

las `Series` de tipo categórico tienen un conjunto de funciones que se puden aplicar exclusivamente a ellas y se pueden consutar [en la documentación de `pandas.Series.cat](https://pandas.pydata.org/docs/reference/api/pandas.Series.cat.html).

In [ ]:
df.genero.cat.categories

Otra manera de generar categorias, es discretizando una variable numérica usando `cut()`, por ejemplo:

In [ ]:
df = pd.DataFrame({"value": np.random.randint(0, 100, 20)})

labels = [f"{i} - {i + 9}" for i in range(0, 100, 10)]

df["group"] = pd.cut(
   df['value'], 
   bins=range(0, 105, 10), 
   right=False, 
   labels=labels
)

df.head(10)

En este caso las categorias vienen ordenadas y se puede revisar su ordenamiento como:

In [ ]:
df['¿viejo?'] = df['group'] > '40 - 49'
df.head(10)

Si queremos definir categorías ordenadas con valores pre establecidos, entonces se puede hacer usando `pandas.Categorical`

In [ ]:
np.random.seed(42)
df = pd.DataFrame(
    {'A': np.random.choice(['a', 'b', 'c', 'd'], 10)}
)
df['B'] = pd.Categorical(df['A'], categories=['c', 'b', 'a'], ordered=True)
df['C'] = df['A'] > 'b'
df['D'] = df['B'] > 'b'
df

### 2. Operaciones en categorias

Se puede ver una variable numérica asociada a una categoría con `Series.cat.code`. Los valores perdidos (categoría no encontrada) son tratados como -1.


In [ ]:
df['B'].cat.codes

Se pueden renombrar los valores de la variable categórica:

In [ ]:
df['B'] = df['B'].cat.rename_categories(
    {'a': 'A', 'b': 'B', 'c': 'C'}
)
df

tambien se pueden agregar nuevos valores a la variable categórica, o reordenar las variables categóricas entre otras funciones.

In [ ]:
df['B'] = df['B'].cat.add_categories(['D'])

In [ ]:
df['B'] = df['B'].cat.reorder_categories(['D', 'C', 'B', 'A'])

### 3. Manipulación básica de categorías

Si bien estos no son únicos para categorías, las dos operaciones básicas para inspeccionar variables categóricas son `unique` y `value_counts`. Evita usarlos en variables numéricas porque va a ser imposible de inspeccionar.

In [ ]:
df['B'].unique()

In [ ]:
df['A'].unique()

In [ ]:
df['B'].value_counts()

In [ ]:
df['A'].value_counts()

Veamos con un ejemplo propuesto en [este blog de DataCamp](https://www.datacamp.com/tutorial/categorical-data) con un conjunto de datos:

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
data.head()

In [ ]:
data['Cut'] = data['Cut'].astype('category')
data['Cut'].value_counts()

Igual así no se ve muy bien, así que se puede hacer una gráfica más suave de inspeccionar:

In [ ]:
import plotly.express as px 
fig = px.bar(data['Cut'].value_counts())
fig.show()

y tambien se puede regrupar respecto a una variable categórica

In [ ]:
data[['Cut', 'Carat Weight', 'Price']].groupby('Cut').mean()

Con variables categóricas (o tipo `object`) se puede estudiar la relación entre dos variables usando `crosstab`:

In [ ]:
pd.crosstab(index=data['Cut'], columns=data['Color'])

O, si se busca combinarlo con otras variables de maneras más complejas, se puede usar la función `pivot_table` como se ve a continuación:

In [ ]:
pd.pivot_table(
    data, 
    values='Price', 
    index='Cut', 
    columns='Color', 
    aggfunc=np.mean
)